# Кластеризация запроса пользователя

Необходимо сделать алгоритм, который кластеризует запросы, выделить с помощью него семантические группы. В итоге по входному списку поисковых запросов должен выдаваться список запросов с кластерами для них по разным семантическим признакам.
Кластеризацию сделать по следующим признакам:
1. по занятости (Фильтр Занятость: подработка, ночная, вечерняя, посменная, вахта и тд.)
2. по должности-лемме (повар, строитель, водитель и тд.)
3. по дополнительному признаку: для инвалидов, для студентов, для школьников, для пенсионеров, для мужчин, для женщин
4. по условиям: с ежедневной оплатой, с проживанием
5. общие фразы про работу (не содержит других признаков)

Одна и та же фраза может попасть в разные кластеры

## Загрузка данных

In [18]:
import pandas as pd


In [16]:
df = pd.read_csv("data/answers.csv", index_col=0)


In [17]:
df

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,фарпост работа владивосток,NaN,NaN,NaN,NaN,общая фраза
1,кофейни вакансии,NaN,NaN,NaN,NaN,общая фраза
2,работа разнорабочие часовой,NaN,Рабочий,NaN,NaN,NaN
3,личный водитель на день,на неполный день,Водитель,NaN,NaN,NaN
4,работа от работодателя персональный водитель,NaN,Водитель,NaN,NaN,NaN
...,...,...,...,...,...,...
14240,япония вакансии,NaN,NaN,NaN,NaN,общая фраза
14241,япония работа,NaN,NaN,NaN,NaN,общая фраза
14242,японский язык работа,NaN,NaN,NaN,NaN,общая фраза
14243,яппи вакансии,NaN,NaN,NaN,NaN,общая фраза


## Model

### TF-IDF + CatBoost

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['query'], df.drop(['query'], axis=1), test_size=0.2, random_state=42)

In [29]:
y_train

,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
4355,Вахта,Вахтер,NaN,NaN,NaN
5406,NaN,Бортпроводник,NaN,NaN,NaN
12357,NaN,NaN,NaN,NaN,общая фраза
5414,NaN,Распространитель,NaN,NaN,NaN
1091,NaN,NaN,NaN,NaN,общая фраза
...,...,...,...,...,...
5191,NaN,"Медсестра, медбрат",NaN,NaN,NaN
13418,NaN,NaN,NaN,NaN,общая фраза
5390,NaN,Охранник,NaN,NaN,NaN
860,Удаленная,Дизайнер,NaN,NaN,NaN


In [39]:
y_train['общие фразы'].fillna(0).map({"общая фраза": 1, 0: 0})

4355     0
5406     0
12357    1
5414     0
1091     1
        ..
5191     0
13418    1
5390     0
860      0
7270     0
Name: общие фразы, Length: 11396, dtype: int64

In [ ]:
y_train['общие фразы']

4355     NaN
5406     NaN
12357      1
5414     NaN
1091       1
        ... 
5191     NaN
13418      1
5390     NaN
860      NaN
7270     NaN
Name: общие фразы, Length: 11396, dtype: object

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score


# Шаг 1: Преобразование текста в TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])  # Преобразуем текст в TF-IDF

# Шаг 2: Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X,  df.drop(['query'], axis=1), test_size=0.2, random_state=42)

# Шаг 3: Обучение модели CatBoost
model = CatBoostClassifier()
y_train_label = y_train['общие фразы'].fillna(0).map({"общая фраза": 1, 0: 0})
y_test_label = y_test['общие фразы'].fillna(0).map({"общая фраза": 1, 0: 0})

model.fit(X_train, y_train_label)

# Шаг 4: Оценка качества модели
y_pred = model.predict(X_test)

# Вывод точности модели
accuracy = accuracy_score(y_test_label, y_pred)
print(f"Точность модели: {accuracy}")


Learning rate set to 0.029118
0:	learn: 0.6837499	total: 23.5ms	remaining: 23.5s
1:	learn: 0.6747908	total: 46ms	remaining: 22.9s
2:	learn: 0.6649415	total: 67.9ms	remaining: 22.6s
3:	learn: 0.6567425	total: 88.2ms	remaining: 21.9s
4:	learn: 0.6492665	total: 109ms	remaining: 21.8s
5:	learn: 0.6432767	total: 129ms	remaining: 21.4s
6:	learn: 0.6365784	total: 151ms	remaining: 21.4s
7:	learn: 0.6318853	total: 171ms	remaining: 21.2s
8:	learn: 0.6282350	total: 192ms	remaining: 21.1s
9:	learn: 0.6240625	total: 247ms	remaining: 24.4s
10:	learn: 0.6201153	total: 278ms	remaining: 25s
11:	learn: 0.6155363	total: 298ms	remaining: 24.5s
12:	learn: 0.6114396	total: 320ms	remaining: 24.3s
13:	learn: 0.6082838	total: 340ms	remaining: 24s
14:	learn: 0.6049190	total: 362ms	remaining: 23.7s
15:	learn: 0.6022728	total: 385ms	remaining: 23.7s
16:	learn: 0.5996730	total: 413ms	remaining: 23.9s
17:	learn: 0.5954361	total: 438ms	remaining: 23.9s
18:	learn: 0.5925036	total: 463ms	remaining: 23.9s
19:	learn: 0.

In [43]:

# Шаг 4: Оценка качества модели
y_pred = model.predict(X_train)

# Вывод точности модели
accuracy = accuracy_score(y_train_label, y_pred)
accuracy

0.9174271674271675

In [44]:
df

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,фарпост работа владивосток,NaN,NaN,NaN,NaN,общая фраза
1,кофейни вакансии,NaN,NaN,NaN,NaN,общая фраза
2,работа разнорабочие часовой,NaN,Рабочий,NaN,NaN,NaN
3,личный водитель на день,на неполный день,Водитель,NaN,NaN,NaN
4,работа от работодателя персональный водитель,NaN,Водитель,NaN,NaN,NaN
...,...,...,...,...,...,...
14240,япония вакансии,NaN,NaN,NaN,NaN,общая фраза
14241,япония работа,NaN,NaN,NaN,NaN,общая фраза
14242,японский язык работа,NaN,NaN,NaN,NaN,общая фраза
14243,яппи вакансии,NaN,NaN,NaN,NaN,общая фраза


In [57]:
df["по условиям"].value_counts(dropna=False)

по условиям
NaN                     13679
с ежедневной оплатой      374
с проживанием             192
Name: count, dtype: int64

In [58]:
df[df["по условиям"] == 'с ежедневной оплатой']

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
46,работа на грузовом авто оплата ежедневно,NaN,NaN,NaN,с ежедневной оплатой,NaN
49,водитель курьер с личным автомобилем ежедневная,NaN,Водитель,NaN,с ежедневной оплатой,NaN
54,работа курьер на авто ежедневные выплаты,NaN,Курьер,NaN,с ежедневной оплатой,NaN
58,вакансии курьер на авто ежедневные выплаты,NaN,Курьер,NaN,с ежедневной оплатой,NaN
64,комплектовщик вакансии с ежедневной оплатой,NaN,Комплектовщик,NaN,с ежедневной оплатой,NaN
...,...,...,...,...,...,...
13907,удаленная работа с еженедельной оплатой,Удаленная,NaN,NaN,с ежедневной оплатой,NaN
13908,удаленная работа с почасовой оплатой,Удаленная,NaN,NaN,с ежедневной оплатой,NaN
13909,удаленная работа с телефона ежедневная оплата,Удаленная,NaN,NaN,с ежедневной оплатой,NaN
13910,удаленная работа с телефона ежедневная оплата ...,Удаленная,NaN,NaN,с ежедневной оплатой,NaN


## Генерация датасета

In [60]:
df


,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,фарпост работа владивосток,NaN,NaN,NaN,NaN,общая фраза
1,кофейни вакансии,NaN,NaN,NaN,NaN,общая фраза
2,работа разнорабочие часовой,NaN,Рабочий,NaN,NaN,NaN
3,личный водитель на день,на неполный день,Водитель,NaN,NaN,NaN
4,работа от работодателя персональный водитель,NaN,Водитель,NaN,NaN,NaN
...,...,...,...,...,...,...
14240,япония вакансии,NaN,NaN,NaN,NaN,общая фраза
14241,япония работа,NaN,NaN,NaN,NaN,общая фраза
14242,японский язык работа,NaN,NaN,NaN,NaN,общая фраза
14243,яппи вакансии,NaN,NaN,NaN,NaN,общая фраза


In [89]:
df['по дополнительному признаку'].unique()

array([nan, 'для мужчин', 'без опыта', 'для женщин', 'для студентов',
       'для женщин,для пенсионеров', 'для инвалидов', 'для пенсионеров',
       'для пенсионеров,для мужчин', 'для мужчин,для женщин',
       'без опыта,для женщин', 'для школьников',
       'без опыта,для студентов', 'для пенсионеров,для женщин',
       'для женщин,без опыта', 'для студентов,для школьников',
       'для школьников,для студентов', 'для пенсионеров,для инвалидов',
       'для студентов,без опыта'], dtype=object)

In [91]:
df[df['по дополнительному признаку'] == 'для женщин,для пенсионеров']

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
448,работа для женщин пенсионеров,NaN,NaN,"для женщин,для пенсионеров",NaN,NaN
7776,работа для женщин пенсионеров во владивостоке,NaN,NaN,"для женщин,для пенсионеров",NaN,NaN
12880,работа охранником в москве по вахте для женщин...,Вахта,Охранник,"для женщин,для пенсионеров",NaN,NaN


In [70]:
job_df = df['по должности-лемме'].value_counts()

In [81]:
df[df['по должности-лемме'] == 'Курьер']

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
8,скачать приложение курьер работа,NaN,Курьер,NaN,NaN,NaN
17,яндекс курьер на авто компании,NaN,Курьер,NaN,NaN,NaN
42,курьер вакансии прямых работодателей,NaN,Курьер,NaN,NaN,NaN
52,курьер с личным легковым автомобилем вакансии,NaN,Курьер,NaN,NaN,NaN
54,работа курьер на авто ежедневные выплаты,NaN,Курьер,NaN,с ежедневной оплатой,NaN
...,...,...,...,...,...,...
14233,яндекс еда курьер,NaN,Курьер,NaN,NaN,NaN
14234,яндекс курьер без термокороба,NaN,Курьер,NaN,NaN,NaN
14235,яндекс курьер доставка,NaN,Курьер,NaN,NaN,NaN
14236,яндекс курьер на авто,NaN,Курьер,NaN,NaN,NaN


In [80]:
job_df[job_df < 500].index.tolist()

['Курьер',
 'Помощник',
 'Вахтер',
 'Модератор',
 'Грузчик',
 'Кладовщик',
 'Уборщик, горничная',
 'Врач',
 'Специалист АХО',
 'Юрист',
 'Медсестра, медбрат',
 'Сварщик',
 'Инженер',
 'Слесарь',
 'Тренер',
 'Дизайнер',
 'Охранник',
 'Монтажник',
 'Администратор',
 'Рабочий',
 'Машинист',
 'Руководитель, начальник',
 'Пивовар',
 'Продавец',
 'Менеджер',
 'Преподаватель',
 'Массажист',
 'Няня',
 'Инструктор',
 'Электрик',
 'Учитель',
 'Швея',
 'Сборщик',
 'Парикмахер',
 'Кухонный работник',
 'Маляр',
 'Сторож',
 'Зоотехник',
 'Электромонтер',
 'Логист',
 'Бортпроводник',
 'Комплектовщик',
 'Переводчик',
 'Токарь',
 'Домработница, домработник',
 'Копирайтер',
 'Механик',
 'Установщик',
 'Программист',
 'Фотограф',
 'Дворник',
 'Сиделка',
 'Психолог',
 'Плотник',
 'Диспетчер',
 'Косметолог',
 'Матрос',
 'Смотритель',
 'Штукатур',
 'Ассистент',
 'Директор',
 'Отделочник',
 'Автоэлектрик',
 'Педагог',
 'Модель',
 'Проектировщик',
 'Оценщик',
 'Технолог',
 'Сметчик',
 'Плиточник',
 'Репетитор